#LiveCon Printer
This is a proof of concept, to directly producting a Program book,
out of the LiveCon CSV.


In [1]:
#CONFIG
import pytz
LOCAL_TIMEZONE = pytz.timezone("Australia/Perth")
SLOT_LENGTH_MINUTES = 30


In [2]:
def normalise(string): 
    import unicodedata
    import pylatex.utils
    import re
    string = string.replace("\r",'')
    string = str(pylatex.utils.escape_latex(string))
    string = re.sub(r'(\\\\)', r'\\', string )
    return string.strip()
    

def load_list(csl):
    ldlst =  str.split(normalise(csl), ', ')
    if ldlst==['']:
        return []
    else:
        return ldlst
    
def parse_datetime(datetime_str):
    import dateutil.parser
    
    raw = dateutil.parser.parse(datetime_str)
    return raw.astimezone(LOCAL_TIMEZONE)


class session (object):
    def __init__(self, id,
                 start_time_str,end_time_str,
                 title,tags_str,people_str,
                 venues_str,description):
        self.id = int(id)
        self.start = parse_datetime(start_time_str)
        self.end = parse_datetime(end_time_str)
        self.title = normalise(title)
        self.tags = load_list(tags_str)
        self.people = load_list(people_str)
        self.venues = load_list(venues_str)
        self.description = normalise(description)
    
    @property
    def day(self):
        week_days   = ['Sunday', 
          'Monday', 
          'Tuesday', 
          'Wednesday', 
          'Thursday',  
          'Friday', 
          'Saturday']
        daynum = self.start.weekday()
        return week_days[daynum]
    
    
    @property
    def start_time(self):
        return self.start.strftime("%H:%M")
    
    @property
    def end_time(self):
        return self.end.strftime("%H:%M")
    
    @property
    def duration(self):
        return self.end - self.start
    

In [3]:
import csv

sessions = []
with open('con_data/swacon_draft0_no15min_2015.csv', 'rU') as csvfile:
    con_csv = csv.reader(csvfile)
    next(con_csv) #Skip heading
    sessions = [session(*row) for row in con_csv]

sessions.sort(key=lambda ss: ss.start)



In [4]:
def write_descriptions(sessions, doc):
    from pagelayout import Multicols
    from itertools import groupby
    with doc.create(Section('Program',numbering=False)):
        with doc.create(Multicols(2)):
            for day_name, day_session in groupby(sessions, lambda ss: ss.day):
                with doc.create(Subsection(day_name,numbering=False)):
                    for session in day_session:
                        with doc.create(Subsubsection(session.title,numbering=False)):
                            with doc.create(Description()) as desc:
                                doc.append('\setlength{\itemsep}{0pt}')
                                doc.append('\setlength{\parsep}{0pt}')
                                doc.append('\setlength{\parskip}{0pt}')
                                desc.additem("Time:", session.start_time+" -- "+session.end_time)

                                if len(session.venues)>0:
                                    desc.additem("Venue:", ", ".join(session.venues))
                                if len(session.people)>0:
                                    desc.additem("People:", ", ".join(session.people))
                                if len(session.tags)>0:
                                    desc.additem("Tags:", ", ".join(session.tags))
                            doc.append(session.description)

In [5]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package
from pylatex.command import Command
from lists import Description

from tabulartimetable import write_timetable


doc = Document(documentclass="scrreprt")
doc.packages.append(Package('geometry', options=['tmargin=1cm','bmargin=1cm',
                                                 'lmargin=1cm','rmargin=1cm',]))

write_timetable(sessions,doc, SLOT_LENGTH_MINUTES)
write_descriptions(sessions,doc)


In [6]:
from IPython.display import FileLink, FileLinks
with open("out/temp.tex", 'w') as temp_out:
    doc.dump(temp_out)

!lualatex --output-directory=out --interaction=nonstopmode temp.tex
FileLinks("./out")

This is LuaTeX, Version beta-0.70.2-2012070300 (TeX Live 2012/Debian)
 restricted \write18 enabled.
(./out/temp.tex
LaTeX2e <2011/06/27>
LuaTeX adaptation of babel <v3.8m-luatex-1.5> and hyphenation patterns for engl
ish, dumylang, nohyphenation, ethiopic, farsi, arabic, pinyin, croatian, bulgar
ian, ukrainian, russian, slovak, czech, danish, dutch, usenglishmax, ukenglish,
 finnish, french, basque, ngerman, german, swissgerman, ngerman-x-2012-05-30, g
erman-x-2012-05-30, monogreek, greek, ibycus, ancientgreek, hungarian, bengali,
 tamil, hindi, telugu, gujarati, sanskrit, malayalam, kannada, assamese, marath
i, oriya, panjabi, italian, latin, latvian, lithuanian, mongolian, mongolianlmc
, nynorsk, bokmal, indonesian, esperanto, coptic, welsh, irish, interlingua, se
rbian, serbianc, slovenian, friulan, romansh, estonian, romanian, armenian, upp
ersorbian, turkish, afrikaans, icelandic, kurmanji, polish, portuguese, galicia
n, catalan, spanish, swedish, thai, loaded.
(/usr/share/texlive

./out/
  temp.log
  temp.tex
  temp.aux
  temp.pdf

In [53]:
round(1.5)

2

In [50]:
from pylatex.base_classes import *
from pylatex.parameters import Arguments
class FixedTextbox(BaseLaTeXClass):#HACK: This should prob not be done by raw text dumping the whole thing...
    def __init__(self,text, hpos,vpos, height,width):
        self.text=text
        self.hpos=hpos
        self.vpos=vpos
        self.height=height
        self.width=width
        
        packages = [Package('textpos', options='absolute'), Package('tcolorbox', options='fitting')]
        BaseLaTeXClass.__init__(self,packages)
    
    def dumps(self): 
        return """
        \\begin{textblock*}{%(width)s}(%(hpos)s, %(vpos)s)
            \\tcboxfit[height=%(height)s]{%(text)s}
        \\end{textblock*}
        """ % self.__dict__
    
class TextcolorboxStyle(BaseLaTeXClass):
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs= kwargs
        packages = [Package('tcolorbox', options='fitting')]
        BaseLaTeXClass.__init__(self, packages=packages)

    def dumps(self): 
        def express_arguments(args,kwargs):
            args_strs = list(map(str,args))
            kwargs_strs = ["%s=%s" % (k,v) for (k,v) in kwargs.items()]
            return ",".join(args_strs + kwargs_strs)
        return "\\tcbset{%s}" % express_arguments(self.args, self.kwargs)
    

In [59]:
def with_units(func):
    def wrapper(*args, **kwargs):
        units = args[0].units #Assumes that this is decorating a method on a class with a units field
        result = func(*args, **kwargs)
        return str(round(result))+units
    return wrapper
    
class timetable_metric_solver():
    def __init__(self,sessions, hour_len, venue_width, units ):
        from venueordering import get_order_from_sessions
        self.hour_len = hour_len
        self.venue_width = venue_width
        self.units = units
        self.venues_to_x = { name:ii*venue_width
                                for ii, name in enumerate(get_order_from_sessions(sessions))}
        
        from itertools import groupby
        self.day_starts = {date:min(map(lambda ss: ss.start, sessions))
                                for date, sessions in groupby(sessions, lambda ss: ss.start.date())}
    
    @with_units
    def get_width(self,session):
        if session.venues:
            return len(session.venues)*self.venue_width    
        else:
            nVenues = len(self.venues_to_x.keys())
            return nVenues*self.venue_width    
        
    @with_units
    def get_x(self, session):
        #Get the left most venue position
        if session.venues:
            return min(map(lambda vv: self.venues_to_x[vv],session.venues))
        else:
            return 0 # fills all Venues, starting from left
            
            
    
    def duration_to_height(self,duration):
        return duration.seconds/(60*60)*self.hour_len
    
    @with_units
    def get_y(self, session):
        day_start = self.day_starts[session.start.date()]
        time_til_start = session.start - day_start
        return self.duration_to_height(time_til_start)
    
    @with_units
    def get_height(self,session):
        return self.duration_to_height(session.duration)
    

In [60]:


def make_pretty_timetable(doc,sessions):
    from pylatex.utils import escape_latex
    from itertools import groupby
    tt_solver = timetable_metric_solver(sessions,1.5,2,'cm')



    doc.append(TextcolorboxStyle('fit algorithm=hybrid*',valign='center', 
                                 colframe='blue!50!black',colback='red!10!white',
                                 boxsep='0pt',top='1mm',bottom='1mm',left='1mm',right='1mm'))
    for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):
        with doc.create(Subsection(escape_latex(str(date)), numbering=False)):
            for session in day_sessions:
                doc.append(FixedTextbox(session.title +'\n %s -- %s ' % (session.start_time, session.end_time),
                                        tt_solver.get_x(session),
                                        tt_solver.get_y(session), 
                                        tt_solver.get_height(session),
                                        tt_solver.get_width(session)))

            doc.append(Command('newpage'))



In [61]:
from IPython.display import FileLink, FileLinks
with open("out/temp.tex", 'w') as temp_out:
    doc.dump(temp_out)

#!lualatex --output-directory=out --interaction=nonstopmode temp.tex
FileLinks("./out")

./out/
  temp.log
  temp.tex
  temp.aux
  temp.pdf